In [4]:
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip, concatenate
from moviepy.video.fx import resize
from moviepy.video.tools.subtitles import SubtitlesClip
import scipy.io.wavfile as wav
import numpy as np
import re


In [23]:
def cropVideo(originalPath, newPath):
    orignal = VideoFileClip(originalPath)
    new_width = int(orignal.size[1] * (9/16)) 
    x_offset = (orignal.size[0] - new_width) / 2
    cropped_clip = orignal.crop(x_offset, 0, x_offset + new_width, orignal.size[1])
    cropped_clip = cropped_clip.subclip(0, 240)
    cropped_clip.write_videofile(newPath, fps=30, codec='libx264')
    cropped_clip.close()

def makeVideo(gameplayVidPath, audioFilePath, finalVidPath):
    TTS = AudioFileClip(audioFilePath)
    gameplayVid = VideoFileClip(gameplayVidPath)
    gameplayVid = gameplayVid.subclip(0,TTS.duration)
    generator = lambda txt: TextClip(txt, font='Arial-Bold', color = 'white', method='caption',fontsize=32, stroke_color='black', stroke_width=1)
    subtitles = SubtitlesClip("./text/subs.srt", generator)
    subtitles = subtitles.set_position(("center","center")).set_duration(TTS.duration)
    final = CompositeVideoClip([gameplayVid, subtitles])
    final.audio = TTS
    final.write_videofile(finalVidPath, fps=30)

def getTextArray(textFile):
    f = open(textFile, "r")
    text = f.read()
    f.close()
    sentence_pattern = r'[.,!?()]\s*'
    sentences = re.split(sentence_pattern, text)
    return sentences[0:len(sentences)-1]

def generateTiming(audioFile, min_silence_duration):
    audio = AudioFileClip(audioFile)
    audio.write_audiofile('audio/TTS.wav') # Converts mp3 into wav file
    audio.close()
    sample_rate, audio_data = wav.read('audio/TTS.wav')

    threshold = 600  # Adjust this value based on your audio data

    silence_timing = []
    is_silence = True
    silence_start_time = 0

    for i, amplitude in enumerate(audio_data):
        if abs(amplitude[0]) > threshold:
            if not is_silence:
                is_silence = True
                silence_duration = (i / sample_rate) - silence_start_time
                if silence_duration >= min_silence_duration:
                    silence_timing.append((silence_start_time, i / sample_rate))
        else:
            if is_silence:
                silence_start_time = i / sample_rate
                is_silence = False

    # # Check if the audio ends with silence longer than min_silence_duration
    if is_silence:
        silence_duration = (len(audio_data) / sample_rate) - silence_start_time
        if silence_duration >= min_silence_duration:
            silence_timing.append((silence_start_time, len(audio_data) / sample_rate))

    sound_segments = []
    for i in range(len(silence_timing) - 1): # Genereates segments of sound from silence
        silence_end_time = silence_timing[i][1]
        next_silence_start_time = silence_timing[i + 1][0]
        sound_segments.append((silence_end_time, next_silence_start_time))
    sound_segments.append((silence_timing[len(silence_timing)-1][1], len(audio_data) / sample_rate)) # Add the last segment
    return sound_segments

def generateSubtitles(sentences, timing):
    f = open("text/subs.srt", "w")
    for i in range(0,len(timing)):
        f.write(str(i+1) + '\n')
        f.write(f'{convertToTime(timing[i][0])} --> {convertToTime(timing[i][1])} \n')
        f.write(sentences[i]+ '\n' + '\n')
    f.close()

def convertToTime(seconds: float): #Conversion from seconds to 00:00:00,000 for SRT file
    millisecond = seconds % 1 * 1000
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return f'{"%02d" %hour}:{"%02d" %minutes}:{"%02d" %seconds},{"%03d" %millisecond}'


In [6]:
cropVideo('videos/minecraft2.mp4', 'videos/minecraftCropped.mp4')

Moviepy - Building video videos/minecraftCropped.mp4.
MoviePy - Writing audio in minecraftCroppedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/minecraftCropped.mp4



Moviepy - Done !
Moviepy - video ready videos/minecraftCropped.mp4


In [24]:
textArray = getTextArray('text/what-is-insurance.txt')
timing = generateTiming('audio/what-is-insurance.mp3' ,0.18)
generateSubtitles(textArray, timing)
makeVideo('videos/minecraftCropped.mp4', 'audio/TTS.wav', 'finalVid/what-is-insurance.mp4')

MoviePy - Writing audio in audio/TTS.wav


chunk:   0%|          | 0/997 [00:00<?, ?it/s, now=None]

MoviePy - Done.


Moviepy - Building video finalVid/what-is-insurance.mp4.
MoviePy - Writing audio in what-is-insuranceTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video finalVid/what-is-insurance.mp4



Moviepy - Done !
Moviepy - video ready finalVid/what-is-insurance.mp4
